# Steps to adapt your resume

1.   Create a markdown version of your resume (Note: ChatGPT can do this for you).
2.   Experiment with different prompt templates that take your markdown resume and a job description and output a new resume in markdown.
3.   Use OpenAI’s Python API to prompt GPT-4o-mini to rewrite your resume dynamically.
4.   Convert the markdown file to HTML and then to PDF with the markdown and pdfkit libraries, respectively.


## Installing libraries

In [3]:
!pip install openai markdown pdfkit
!apt-get install -y wkhtmltopdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1
  libdaemon0 libevdev2 libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0 libmbim-glib4 libmbim-proxy
  libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 l

In [4]:
import markdown
import pdfkit
import openai

## Reading parameters

In [5]:
import yaml
def read_parameters_file(file):
  with open(file, encoding = 'utf-8') as file:
    parameters = yaml.load(
        file,
        Loader = yaml.FullLoader,
    )
    file.close()
    return parameters
parameters = read_parameters_file('parameters.yaml')

from openai import OpenAI

client = OpenAI(
    api_key = parameters['api_key'],
  organization=parameters['organization'],
  project=parameters['project'],
)


## A function that adapts the passed resume to the given job description

In [6]:
def adapt_resume(md_resume, job_desciption):

    # prompt (assuming md_resume and job_desciption have been defined)
    prompt = f"""
    I have a resume formatted in Markdown and a job description. \
    Please adapt my resume to better align with the job requirements while \
    maintaining a professional tone. Tailor my skills, experiences, and \
    achievements to highlight the most relevant points for the position. \
    Ensure that my resume still reflects my unique qualifications and strengths \
    but emphasizes the skills and experiences that match the job description.

    ### Here is my resume in Markdown:
    {md_resume}

    ### Here is the job description:
    {job_desciption}

    Please modify the resume to:
    - Use keywords and phrases from the job description.
    - Adjust the bullet points under each role to emphasize relevant skills and achievements.
    - Make sure my experiences are presented in a way that matches the required qualifications.
    - Maintain clarity, conciseness, and professionalism throughout.

    Return the updated resume in Markdown format.

    """

    # make api call
    response = client.chat.completions.create(
      model= "gpt-4o-mini",#"gpt-3.5-turbo", #
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
      ],
      temperature = 0.25
    )

    # extract response
    resume = response.choices[0].message.content
    return resume

## Reading the resume and job description (Resume is already in markdown format)

In [22]:
# Open the file in read mode
with open('OriginalCV_md.txt', 'r') as file:
    CV_markdown = file.read()

# Open the file in read mode
with open('job_description.txt', 'r') as file:
    job_description = file.read()

## Adapting resume to the job description

In [23]:
adaptedCV_md = adapt_resume(CV_markdown, job_description)

## Converting the markdown adapted resume to HTML and then PDF

In [24]:
filename = 'CV_gpt4omini'

In [25]:
# Convert markdown text to HTML
html_content = markdown.markdown(adaptedCV_md)

with open(f'{filename}.html', 'w') as html_file:
    html_file.write(html_content)


In [26]:
# Convert HTML to pdf

pdfkit.from_file(f'{filename}.html', f'{filename}.pdf')

print("PDF generated successfully!")

PDF generated successfully!
